# Trading Strategy Backtesting with `python_trade_runner`

This notebook demonstrates how to use the `python_trade_runner` library to backtest a trading strategy. We'll go through the steps of importing data, defining a strategy, running a backtest, and analyzing the results.

---

## Table of Contents

1. [Installation](#installation)
2. [Import Required Modules](#import-required-modules)
3. [Import Your Data](#import-your-data)
4. [Define Your Strategy](#define-your-strategy)
5. [Backtest Your Strategy](#backtest-your-strategy)
6. [Analyze Results](#analyze-results)

---

## 1. Installation

First, make sure you've cloned the repository and installed the required dependencies.

In [ ]:
%pip install -r requirements.txt

## 2. Import Required Modules

Import all necessary modules from the `python_trade_runner` package.

In [ ]:
from datetime import datetime, timezone
from python_trade_runner import *

## 3. Import Your Data

Ensure your financial data is in the correct format and located in the appropriate directory. The data should be in CSV format with the following columns:

- `timestamp`, `open`, `high`, `low`, `close`, `volume`

The path structure should be:

```
{PATH_TO_DATA}/{exchange}/{symbol}/{exchange}-{symbol}-{interval}.csv
```

You can set your custom data folder by overwriting `PATH_TO_DATA`.

In [ ]:
# Example: Set custom data path if needed
# PATH_TO_DATA = '/path/to/your/data'

## 4. Define Your Strategy

### Create a Trading Pattern

Define a list of patterns to design your trading strategy. Below is an example of defining a double bottom pattern.

In [ ]:
pattern_list = []
pattern_list.append(AddPoint(0, add_last_datetime))
pattern_list.append(AddPoint(0, add_uptrend, 0, True))
pattern_list.append(AddPoint(0, add_downtrend, 0, True))
pattern_list.append(AddPoint(0, add_uptrend, 0, True))
pattern_list.append(AddPoint(0, add_downtrend, 0, True))
pattern_list.append(FilterPoint(check_highs, 2, 0))
pattern_list.append(FilterPoint(check_highs, 4, 2))
pattern_list.append(FilterPoint(check_lows, 1, 3))

### Create a Trade Builder

A trade builder calculates the trade position based on your pattern, including entry, stop loss, and profit levels.

In [ ]:
trade_builder = TradeBuilder(
    [(2, 1, 1)],  # Entry Level: Max price at point C
    [(1, 2, 1)],  # Stop Level: Min price at point B
    [(1, 2, -1), (2, 1, 2)],  # Profit Level: Calculated for 2:1 reward-to-risk ratio
    market_entry=True,
    max_trade_duration_params=[0, 3, 0.5],
    visual_price_index=[1, 2, 1, 2, 1]
)

#### Visual Representation of the Setup

To visualize the pattern and trade setup, refer to the image below:

![Setup example](https://raw.githubusercontent.com/LoicL971/python_trade_runner/main/setup.png)

## 5. Backtest Your Strategy

### Initialize Backtest Object

Set up the backtest parameters, including the time frame, exchange, symbol, and interval.

In [ ]:
setup = Setup(0, pattern_list, trade_builder)

start = datetime(2022, 6, 15, tzinfo=timezone.utc)
end = datetime(2022, 6, 21, tzinfo=timezone.utc)
exchange = Exchange.BINANCE
symbol = Symbol.BTCUSDT
interval = Interval.M5

backtest = Backtest(
    setup_list=[setup],
    risk_for_setup_id={0: 0.01},
    start_datetime=start,
    end_datetime=end,
    window_size=150,
    initial_balance=1000,
    exchange=exchange,
    symbol=symbol,
    interval=interval,
    test_fees=False
)

### Run the Backtest

In [ ]:
backtest.initialize_chart()
backtest.step_until_end()

## 6. Analyze Results

After running the backtest, you can analyze the results using the following code:

In [ ]:
results = backtest.get_results()

print(f"Winrate: {results.get_winrate():.2%}")
print(f"R ratio: {results.get_r()}")
print(f"Max drawdown: {results.get_max_drawdown():.2%}")
print(f"Expected profit per trade: {results.get_perf():.2%}")
print(f"Succeeded trades: {results.get_nb_success()}")
print(f"Stopped trades: {results.get_nb_stopped()}")
print(f"Canceled trades: {results.get_nb_canceled()}")
print(f"Final Balance: {backtest.balance:.2f}")

**Sample Output:**

```
Winrate: 58.00%
R ratio: 1.0
Max drawdown: 36.48%
Expected profit per trade: 1.94%
Succeeded trades: 509
Stopped trades: 370
Canceled trades: 408
Final Balance: 1119.75
```

### Visualizations

#### Plot Balances Over Time

In [ ]:
results.plot_balances()

This will display a graph of your balance over time during the backtest.

![Plotted balances](https://raw.githubusercontent.com/LoicL971/python_trade_runner/main/balances.png)

#### Show Specific Trades

You can visualize specific trades by their index:

In [ ]:
results.show_some_archieved_trades(indexes=[25, 26])

This will display charts of the specified trades.

![Plotted trade](https://raw.githubusercontent.com/LoicL971/python_trade_runner/main/trade.png)

## Conclusion

You've successfully run a backtest using the `python_trade_runner` library! This notebook provides a template for testing your own trading strategies. Feel free to modify the pattern and trade builder to suit your specific needs.